Michael Toolin<br>
Natural Language Processing<br>
DS 7337-403 <br>
Homework 5

In [1]:

""""
Created on Thu Feb 28 19:30:48 2019

@author: mtool
"""
import sys; print ("Python", sys.version)
from nltk.corpus import stopwords
import nltk
from nltk import Tree
from nltk import word_tokenize, pos_tag,ne_chunk
from nltk import RegexpParser

from pattern.en import tag, parsetree, Chunk
from bs4 import BeautifulSoup 
import requests
from difflib import SequenceMatcher
import re
import numpy as np

nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

Python 3.6.6 |Anaconda custom (64-bit)| (default, Jun 28 2018, 11:27:44) [MSC v.1900 64 bit (AMD64)]


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\mtool\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\mtool\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\mtool\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

### Movie genre selection and high level algorithm approach
I chose to examine reviews for the top 240 horror movies listed on the IMDB site. The following steps were used to scrape each page and find the permalink and eventually the review text for each movie.<br>
 * I used the URL of this website to scrape the URL's for each individual movie.  These URLs are savedin a dictionary containing **{title:movieURL}**<br><br>
 * Within each movies page, I noticed the difference between the movies's main page and the user review page was the attribute on the end of the URL. All movie review pages had the same attribute. I created a new dictionary saving the user review URL **{title:reviewURL}**.<br><br>
 * Within each user review page a pattern was found identifying the permalink for each user review.<br> The review was downloaded and saved in it's own dictionary **{title:reviewText}**.<br><br>
 * Once the review text was gathered, each text was run through the shallow parser in *pattern.en* called *parsetree*.<br> The chunks were filtered to only retrieve the noun phrase chunks.  These chunks were put in a list and stored in a dictionary with the format **{title:[chunk1, chunk2,...,chunkn]}**.<br><br>
 
 #### Get the URL for all 240 movies and save in dictionary titled reviewDict{}


In [2]:
#%%
#
# Top 240 horror movies @ https://www.imdb.com/list/ls059633855/
#
debug = False
pages = ['1', '2', '3']
reviewAttrb = 'reviews?ref_=tt_ql_3'
movieAttrb = '?ref_=ttls_li_tt'
reviewDict = {}

#
# Get all URL's from page of 240 horror movies listing.
# Each movie has multiple pages, but we want the one that has the
# attribute ?ref_=ttls_li_tt.  Once we find these, get the title of the
# movie and then update the URL to point to the review page by replacing
# the attribute with reviews?ref_=tt_ql_3
# Create a dictionary of the movie titles and the URLs to user review pages
# Note this URL points to the page with ALL the reviews for the movie
#
# I noticed the page with user reviews for each movie uses the same URL
# but just passes a different attribute after the movie number. I don't have
# to go to each movie page to find the review page, just subst the attribute
# at the end to get to the user review pages
#
for pgNum in pages:
    urlStr='https://www.imdb.com/list/ls059633855/?sort=list_order,asc&st_dt=&mode=detail&page='+pgNum
    page = requests.get(urlStr)
    soup = BeautifulSoup(page.text,'html.parser')
    print("Getting movie page URL's in page {}".format(pgNum))
    for a in soup.find_all('a', href=re.compile('/?ref_=ttls_li_tt')):
        if debug:
            print ("Found the URL:", a['href'],'for the movie', a.contents[0])
        revURL = a['href'].replace(movieAttrb, reviewAttrb)
        revURL = 'https://www.imdb.com'+ revURL
        title = str(a.contents[0])
        reviewDict [title] = revURL
print('\nTotal of URLs found = ', len(reviewDict.keys()))

Getting movie page URL's in page 1
Getting movie page URL's in page 2
Getting movie page URL's in page 3

Total of URLs found =  0


#### Find the PermaLink URL for the first review for each movie
Store the PermaLink URL in the dictionary named permaDict{title:permaURL}

In [3]:
#%%
#
# Find first permaLink and create a dictionary 
# {Movie title:permaLink} This page has movie rankings by user, etc...
#
permaDict ={}
print ("Getting PermaLinks...\n")
for k in reviewDict:
    reviewURL = reviewDict.get(k)
    title = str(k)                             # Save title for key in new Dict
    reviewPage = requests.get(reviewURL)
    reviewSoup = BeautifulSoup(reviewPage.text, 'html.parser')
    permaTag = reviewSoup.find('a', href=re.compile('/review/rw')) 
    if debug:
        print('PermaLink review URL:', permaTag['href'],'for movie',k)  
    permaURL = 'https://www.imdb.com'+ permaTag['href']
    permaDict [title] = permaURL      

Getting PermaLinks...



#### Retrieve the body of each review and store in a dictionary textDict{title:reviewText}

In [4]:
#%%
#
# Retrieve the body of each each review and store in a dictionary
# {Title: Review Text}
#
textDict = {}
print('Getting review text... \n')
for k1 in permaDict:
    rawReviewText =''                           # Clear the buffer
    permaURL = permaDict.get(k1)
    title = str (k1)                            # Keep track of the movie title
    permaPage = requests.get(permaURL)
    permaSoup = BeautifulSoup(permaPage.text, 'html.parser')
    permaTag = permaSoup.find('div', {'class':'text show-more__control'})
    [p.replace_with(' ') for p in permaTag.findAll('br')]
    if debug:
        print(permaURL)
        print('Movie',k1, 'has text',permaTag.contents[0])
    for t in permaTag:
        rawReviewText = rawReviewText+ str(t)
    textDict[title] = rawReviewText

Getting review text... 



#### Parse each review and store all noun phrases as chunks in a dictionary nounPhraseDict{title:[chunk1, chunk2,...,chunkn]}
The pattern.en parser does a very good job of finding proper names and using them in the noun phrases.  I performed a cursory examination of the noun phrases and found many proper names used in the right context.  I didn't feel I needed to add names to the working lexicon.

In [5]:
#%%
#
# Now that all the reviews are gathered, chunk each one and create a
# dictionary with {Title:[(chunk1, chunk2,...,chunkn)]}
#
nounPhraseDict = {}
for k2 in textDict:
    rawText = textDict.get(k2)
    str_tokens = nltk.word_tokenize(rawText)   # not sure if we need this.
    tree = parsetree(rawText)
    print ('\n',k2)
    for sentence_tree in tree:
        for chunk in sentence_tree.chunks:
            if (chunk.type =='NP'):
                nounPhrase = [(word.string, word.type) for word in chunk.words]
                print("NP ->",nounPhrase)
                nounPhraseDict.setdefault(k2, []).append(nounPhrase)


 The Wailing
NP -> [('SPOILER', 'NN')]
NP -> [('a', 'DT'), ('lot', 'NN')]
NP -> [('films', 'NNS')]
NP -> [('a', 'DT'), ('lot', 'NN')]
NP -> [('films', 'NNS')]
NP -> [('two', 'CD'), ('and', 'CC'), ('a', 'DT'), ('half', 'NN'), ('hour', 'NN'), ('technically-perfect', 'NN')]
NP -> [('humanistic', 'JJ'), ('horror', 'NN'), ('film', 'NN')]
NP -> [('the', 'DT'), ('finest', 'JJS'), ('writer/directors', 'NNS')]
NP -> [('the', 'DT'), ('business', 'NN')]
NP -> [('auteur', 'NN')]
NP -> [('I', 'PRP')]
NP -> [('THE', 'DT'), ('DEVIL', 'NN')]
NP -> [('something', 'NN')]
NP -> [('The', 'DT'), ('closest', 'JJS'), ('analog', 'NN'), ('I', 'PRP')]
NP -> [('David', 'NNP'), ('Lynch', 'NNP')]
NP -> [('Mulholland', 'NNP'), ('Drive', 'NNP')]
NP -> [('suspense', 'NN'), ('opus', 'NN')]
NP -> [('you', 'PRP')]
NP -> [('its', 'PRP$'), ('full', 'JJ'), ('length', 'NN')]
NP -> [('you', 'PRP')]
NP -> [('the', 'DT'), ('theatre', 'NN')]
NP -> [('your', 'PRP$'), ('head', 'NN')]
NP -> [('you', 'PRP')]
NP -> [('Both', 'DT'),


 The Conjuring
NP -> [('the', 'DT'), ('director', 'NN')]
NP -> [('the', 'DT'), ('most', 'RBS'), ('inventive', 'JJ'), ('horror', 'NN'), ('filmmaker', 'NN'), ('working', 'NN')]
NP -> [('the', 'DT'), ('industry', 'NN')]
NP -> [('James', 'NNP'), ('Wan', 'NNP')]
NP -> [('us', 'PRP')]
NP -> [('the', 'DT'), ('retro', 'JJ'), ('days', 'NNS')]
NP -> [('horror', 'NN')]
NP -> [('visually', 'RB'), ('striking', 'JJ'), ('horror', 'NN'), ('film', 'NN')]
NP -> [('the', 'DT'), ('classics', 'NNS')]
NP -> [('no', 'DT'), ('sex', 'NN')]
NP -> [('The', 'DT'), ('Conjuring', 'NNP')]
NP -> [('its', 'PRP$')]
NP -> [('scares', 'NNS')]
NP -> [('The', 'DT'), ('Conjuring', 'NN')]
NP -> [('the', 'DT'), ('married', 'JJ'), ('paranormal', 'JJ'), ('researchers', 'NNS'), ('Ed', 'NNP')]
NP -> [('Patrick', 'NNP'), ('Wilson', 'NNP')]
NP -> [('Lorraine', 'NNP')]
NP -> [('Vera', 'NNP'), ('Farmiga', 'NNP')]
NP -> [('Warren', 'NNP')]
NP -> [('colleges', 'NNS')]
NP -> [('the', 'DT'), ('US', 'NNP')]
NP -> [('all', 'PDT'), ('the',

NP -> [('something', 'NN')]
NP -> [('an', 'DT'), ('apartment', 'NN'), ('block', 'NN')]
NP -> [('the', 'DT'), ('movie', 'NN')]
NP -> [('the', 'DT'), ('sense', 'NN')]
NP -> [('unease', 'NN')]
NP -> [('way', 'NN')]
NP -> [('terror', 'NN')]
NP -> [('it', 'PRP')]
NP -> [('movies', 'NNS')]
NP -> [('The', 'DT'), ('Last', 'JJ'), ('Broadcast', 'NNP')]
NP -> [('The', 'DT'), ('Blair', 'NNP'), ('Witch', 'NNP'), ('Project', 'NN')]
NP -> [('it', 'PRP')]
NP -> [('a', 'DT'), ('way', 'NN')]
NP -> [('those', 'DT'), ('movies', 'NNS')]
NP -> [('thanks', 'NNS')]
NP -> [('some', 'DT'), ('brilliant', 'JJ'), ('set-pieces', 'NNS')]
NP -> [('It', 'PRP')]
NP -> [('the', 'DT'), ('UK', 'NNP')]
NP -> [('Hollywood', 'NNP')]
NP -> [('it', 'PRP')]
NP -> [('Quarantine', 'NN')]
NP -> [('yourself', 'PRP'), ('a', 'DT'), ('favor', 'NN')]
NP -> [('an', 'DT'), ('evening', 'NN')]
NP -> [('You', 'PRP')]
NP -> [('it', 'PRP')]
NP -> [('the', 'DT'), ('best', 'JJS'), ('possible', 'JJ'), ('way', 'NN')]

 The VVitch: A New-England F

NP -> [('it', 'PRP')]
NP -> [('a', 'DT'), ('Hollywood', 'NNP'), ('slasher', 'NN'), ('flick', 'NN')]
NP -> [('this', 'DT'), ('movie', 'NN')]
NP -> [('The', 'DT'), ('women', 'NNS')]
NP -> [('scantily', 'RB'), ('clad', 'JJ'), ('models', 'NNS')]
NP -> [('forests', 'NNS')]
NP -> [('men', 'NNS')]
NP -> [('masks', 'NNS')]
NP -> [('nothing', 'NN')]
NP -> [('these', 'DT'), ('scenes', 'NNS')]
NP -> [('they', 'PRP')]
NP -> [('the', 'DT'), ('viewer', 'NN')]
NP -> [('they', 'PRP')]
NP -> [('they', 'PRP')]
NP -> [('mind', 'NN')]
NP -> [('very', 'RB'), ('dark', 'JJ'), ('places', 'NNS')]
NP -> [('It', 'PRP')]
NP -> [('us', 'PRP'), ('a', 'DT'), ('peek', 'NN')]
NP -> [('the', 'DT'), ('madness', 'NN')]
NP -> [('that', 'WDT'), ('every', 'DT'), ('man', 'NN')]
NP -> [('its', 'PRP$'), ('punches', 'NNS')]
NP -> [('This', 'DT'), ('brutal', 'JJ'), ('realism', 'NN')]
NP -> [('people', 'NNS')]
NP -> [('negative', 'JJ'), ('reviews', 'NN')]
NP -> [('I', 'PRP')]
NP -> [('a', 'DT'), ('movie', 'NN')]
NP -> [('the', 'D

NP -> [('this', 'DT'), ('movie', 'NN')]
NP -> [('He', 'PRP')]
NP -> [('people', 'NNS')]
NP -> [('it', 'PRP')]
NP -> [('he', 'PRP')]
NP -> [('the', 'DT'), ('people', 'NNS')]
NP -> [('it', 'PRP')]
NP -> [('He', 'PRP')]
NP -> [('the', 'DT'), ('modern', 'JJ'), ('day', 'NN'), ('equivalent', 'NN')]
NP -> [('Victor', 'NNP'), ('Frankenstein', 'NNP')]
NP -> [('He', 'PRP')]
NP -> [('he', 'PRP')]
NP -> [('a', 'DT'), ('monster', 'NN')]
NP -> [('it', 'PRP')]
NP -> [('it', 'PRP')]
NP -> [('the', 'DT'), ('world', 'NN')]
NP -> [('Martyrs', 'NNP')]
NP -> [('a', 'DT'), ('monster', 'NN')]
NP -> [('It', 'PRP')]
NP -> [('a', 'DT'), ('ground', 'NN'), ('breaking', 'NN')]
NP -> [('brave', 'NN')]
NP -> [('piece', 'NN')]
NP -> [('work', 'NN')]
NP -> [('audiences', 'NNS'), ('everywhere', 'RB'), ('it', 'PRP')]
NP -> [('the', 'DT'), ('chance', 'NN')]
NP -> [('a', 'DT'), ('genre', 'NN'), ('event', 'NN')]
NP -> [('Frightfest', 'NNP'), ('the', 'DT'), ('audience', 'NN'), ('reaction', 'NN')]
NP -> [('it', 'PRP')]
NP ->

...

NP -> [('It', 'PRP')]
NP -> [('Circus', 'NNP'), ('Maximus', 'NNP')]

 Insidious: Chapter 2
NP -> [('Chapter', 'NN')]
NP -> [('the', 'DT'), ('sequel', 'NN')]
NP -> [('the', 'DT'), ('2011', 'CD'), ('hit', 'NN')]
NP -> [('the', 'DT'), ('same', 'JJ'), ('guy', 'NN')]
NP -> [('James', 'NNP'), ('Wan', 'NNP')]
NP -> [('This', 'DT'), ('is...um...less', 'NN')]
NP -> [('the', 'DT'), ('first...but', 'JJ'), ('I', 'PRP')]
NP -> [('it', 'PRP')]
NP -> [('me', 'PRP')]
NP -> [('the', 'DT'), ('story', 'NN')]
NP -> [('the', 'DT'), ('events', 'NNS')]
NP -> [('the', 'DT'), ('father', 'NN')]
NP -> [('Josh', 'NNP')]
NP -> [('Patrick', 'NNP'), ('Wilson', 'NNP'), ('rescues', 'NNS'), ('his', 'PRP$'), ('son', 'NN')]
NP -> [('a', 'DT'), ('supernatural', 'JJ'), ('ability', 'NN')]
NP -> [('different', 'JJ'), ('worlds', 'NNS')]
NP -> [('He', 'PRP')]
NP -> [('the', 'DT'), ('dark', 'JJ'), ('spirit', 'NN'), ('world', 'NN')]
NP -> [('demons', 'NNS')]
NP -> [('Josh', 'NNP')]
NP -> [('his', 'PRP$'), ('son', 'NN')]
NP -> [(

NP -> [('It', 'PRP')]
NP -> [('the', 'DT'), ('Open', 'NNP'), ('Water/Black', 'NNP'), ('Water/The', 'NNP'), ('Canyon', 'NNP'), ('school', 'NN')]
NP -> [('survival', 'NN'), ('horror', 'NN')]
NP -> [('those', 'DT'), ('films', 'NNS')]
NP -> [('intense', 'JJ'), ('portraits', 'NNS')]
NP -> [('people', 'NNS')]
NP -> [('some', 'DT'), ('form', 'NN')]
NP -> [('wilderness', 'NN'), ('nightmare', 'NN')]
NP -> [('little', 'JJ'), ('hope', 'NN')]
NP -> [('escape', 'NN')]
NP -> [('my', 'PRP$'), ('pick', 'NN')]
NP -> [('the', 'DT'), ('bunch', 'NN')]
NP -> [('one', 'CD'), ('point', 'NN')]
NP -> [('the', 'DT'), ('trio', 'NN')]
NP -> [('predicament', 'NN')]
NP -> [('I', 'PRP')]
NP -> [('I', 'PRP')]
NP -> [('my', 'PRP$'), ('hands', 'NNS')]
NP -> [('they', 'PRP')]
NP -> [('it', 'PRP')]
NP -> [('the', 'DT'), ('tension', 'NN')]
NP -> [('me', 'PRP')]
NP -> [('The', 'DT'), ('characters', 'NNS')]
NP -> [('me', 'PRP')]
NP -> [('the', 'DT'), ('film', 'NN')]
NP -> [('I', 'PRP')]
NP -> [('them', 'PRP')]
NP -> [('Emma

NP -> [('their', 'PRP$'), ('part', 'NN'), ('and', 'CC'), ('they', 'PRP')]
NP -> [('characters', 'NNS')]
NP -> [('They', 'PRP')]
NP -> [('I', 'PRP')]
NP -> [('this', 'DT'), ('movie', 'NN')]
NP -> [('you', 'PRP')]
NP -> [('me', 'PRP'), ('and', 'CC'), ('love', 'NN'), ('movies', 'NNS')]
NP -> [('The', 'DT'), ('Sixth', 'NNP'), ('Sense', 'NN')]
NP -> [('Shutter', 'NNP'), ('Island', 'NNP')]
NP -> [('Inception', 'NNP')]
NP -> [('Dream', 'NNP'), ('House', 'NNP')]
NP -> [('etc', 'NN')]
NP -> [('I', 'PRP')]
NP -> [('you', 'PRP')]
NP -> [('this', 'DT'), ('movie', 'NN')]
NP -> [('You', 'PRP')]
NP -> [('me', 'PRP')]
NP -> [('it', 'PRP'), ('the', 'DT'), ('2nd', 'CD'), ('time', 'NN')]
NP -> [('any', 'DT'), ('clues', 'NNS'), ('you', 'PRP')]
NP -> [('you', 'PRP')]
NP -> [('15', 'CD'), ('minutes', 'NNS')]
NP -> [('movie', 'NN')]
NP -> [('Out', 'NNP')]
NP -> [('700', 'CD'), ('movies', 'NNS')]
NP -> [('my', 'PRP$'), ('collection', 'NN')]
NP -> [('a', 'DT'), ('10', 'CD'), ('star', 'JJ'), ('rating', 'NN')]
N

NP -> [('things', 'NNS')]
NP -> [('you', 'PRP')]
NP -> [('they', 'PRP')]
NP -> [('I', 'PRP')]
NP -> [('the', 'DT'), ('video', 'JJ'), ('game', 'NN'), ('plot--it', 'NN')]
NP -> [('the', 'DT'), ('gamers', 'NNS')]
NP -> [('it', 'PRP')]
NP -> [('Everything', 'NN')]
NP -> [('it', 'PRP')]
NP -> [('me', 'PRP')]
NP -> [('parts', 'NNS')]
NP -> [('the', 'DT'), ('underlying', 'JJ'), ('meanings', 'NNS')]
NP -> [('symbolism', 'NN')]
NP -> [('The', 'DT'), ('characters', 'NNS')]
NP -> [('the', 'DT'), ('acting', 'NN')]
NP -> [('I', 'PRP')]
NP -> [('I', 'PRP'), ('actually', 'RB'), ('FELT', 'NNP')]
NP -> [('them', 'PRP')]
NP -> [('an', 'DT'), ('element', 'NN')]
NP -> [('horror', 'NN'), ('films', 'NNS')]
NP -> [('The', 'DT'), ('procession', 'NN')]
NP -> [('events', 'NNS')]
NP -> [('times', 'NNS')]
NP -> [('the', 'DT'), ('characters', 'NNS')]
NP -> [('I', 'PRP')]
NP -> [('that', 'DT'), ('everything', 'NN')]
NP -> [('the', 'DT'), ('name', 'NN')]
NP -> [('The', 'DT'), ('lighting', 'NN')]
NP -> [('this', 'DT'

NP -> [('the', 'DT'), ('real', 'JJ'), ('gem', 'NN')]
NP -> [('Shiver', 'NN')]
NP -> [('DVD', 'NNP')]
NP -> [('Screw', 'NNP'), ('Hollywood', 'NNP')]
NP -> [('Spain', 'NNP-LOC'), ('a', 'DT'), ('chance', 'NN')]
NP -> [('you', 'PRP')]
NP -> [('You', 'PRP')]
NP -> [('another', 'DT'), ('film', 'NN')]
NP -> [('this', 'DT'), ('year', 'NN')]
NP -> [('such', 'JJ'), ('good', 'JJ'), ('sense', 'NN')]
NP -> [('camera', 'NN'), ('use', 'NN'), ('and', 'CC'), ('the', 'DT'), ('proper', 'JJ'), ('story', 'NN')]
NP -> [('it', 'PRP')]

 Freddy vs. Jason
NP -> [('Evil', 'NNP'), ('vs', 'NNP'), ('Evil', 'NNP'), ('two', 'CD'), ('favorite', 'JJ'), ('slasher', 'NN'), ('icons', 'NNS')]
NP -> [('each', 'DT'), ('other', 'JJ'), ('Freddy', 'NNP'), ('vs', 'NNP'), ('Jason', 'NNP')]
NP -> [('an', 'DT'), ('epic', 'NN'), ('battle', 'NN'), ('fight', 'NN')]
NP -> [('It', 'PRP')]
NP -> [('my', 'PRP$'), ('personal', 'JJ'), ('favorite', 'JJ'), ('horror', 'NN'), ('movies', 'NNS')]
NP -> [('Two', 'CD'), ('titans', 'NNS')]
NP -> [(